In [1]:
# Marta Matice Valdman 1 - solution
# here is the original file in OneDrive
import scipy.io as io
import numpy as np


# from scipy.io import loadmat  # this is the SciPy module that loads mat-files
import matplotlib.pyplot as plt
from datetime import datetime, date, time
import pandas as pd

import numpy.linalg as la
import scipy.linalg as spla

import time
from multiprocessing import Pool, cpu_count
# from functools import partial
import collections
import os
import math

n_processors = cpu_count()

# read file
file = 'mesh_2D_level1.mat'
coordinates = 'coords'
elements = 'elems'

mesh = mesh_coordinates = io.loadmat(file)


In [8]:
"""
mesh structure in Matlab
              dim: 2
      nodes2coord: [12545×2 double]
      nodes2elems: [12545×6 double]
      elems2nodes: [24576×3 double]
    nodesBoundary: [512×1 double]
    nodesInternal: [12033×1 double]
        nodesFree: [12033×1 double]
      Grad1_elems: [24576×3 double]
      Grad2_elems: [24576×3 double]
            areas: [24576×1 double]
             Hstr: [12545×12545 double]
"""
mesh_python = {"dim": mesh['mesh'][0][0][0],
                "nodes2coord": mesh['mesh'][0][0][1],
                "nodes2elems": mesh['mesh'][0][0][2],
                "elems2nodes": mesh['mesh'][0][0][3],
                "nodesBoundary": mesh['mesh'][0][0][4],
                "nodesInternal": mesh['mesh'][0][0][5],
                "nodesFree": mesh['mesh'][0][0][6],
                "Grad1_elems": mesh['mesh'][0][0][7],
                "Grad2_elems": mesh['mesh'][0][0][8],
                "areas": mesh['mesh'][0][0][9],
                "Hstr": mesh['mesh'][0][0][10]}


In [2]:
mesh_coordinates = mesh_python['nodes2coord']
mesh_elements = mesh_python['elems2nodes']
n_n = mesh_coordinates.shape[0]        # number of nodes
n_e = mesh_elements.shape[0]           # number of element
dim = mesh_coordinates.shape[1]        # dimension
print(f'no.nodes: {n_n}, no.elements: {n_e}, dim: {dim}')


no.nodes: 21, no.elements: 24, dim: 2


In [4]:

# print(mesh_coordinates.shape)
# print(mesh_elements.shape)

# coords 3D matice
# single process

start_time = time.time()

coords3D = np.zeros((dim, dim, n_e))
det_coords3D = np.zeros(n_e)
inv_coords3D = np.zeros((dim, dim, n_e))

for el in range(0, n_e):
    nodes = mesh_elements[el]
    coordinates = mesh_coordinates[nodes - 1, :]
    for comp in range(dim):
        coords3D[comp, :, el] = coordinates[comp + 1, :] - coordinates[0, :]

    det_coords3D[el] = np.linalg.det(coords3D[:, :, el])
    inv_coords3D[:, :, el] = np.linalg.inv(coords3D[:, :, el])

print(det_coords3D)
print(sum(det_coords3D) / 6)
print("--single process - %s seconds ---" % (time.time() - start_time))


[0.25 0.25 0.25 0.25 0.25 0.25 0.25 0.25 0.25 0.25 0.25 0.25 0.25 0.25
 0.25 0.25 0.25 0.25 0.25 0.25 0.25 0.25 0.25 0.25]
1.0
--single process - 0.049840450286865234 seconds ---
